<a href="https://colab.research.google.com/github/jinkyukim-me/RNN_Tutorial_Master/blob/master/Simple_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Preparing Data

In [0]:
import torch
from torchtext import data

* Set the random seeds for reproducibility

In [0]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

* TEXT field to define how the review should be processed, and the LABEL field to process the sentiment

In [0]:
TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)

* Load the IMDb dataset.
* Splits it into the canonical train/test splits as torchtext.datasets objects.<br>
* The IMDb dataset consists of 50,000 movie reviews, each marked as being a positive or negative review.

In [0]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:07<00:00, 11.6MB/s]


* See how many examples are in each split by checking their length.

In [0]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


* Check an example.

In [0]:
print(vars(train_data.examples[0]))

{'text': ['Soapdish', 'may', 'go', 'down', 'as', 'one', 'of', 'the', 'single', 'most', 'under', '-', 'rated', 'movies', 'ever', 'made.<br', '/><br', '/>A', 'stellar', ',', 'unselfish', 'cast', 'who', 'understood', 'exactly', 'where', 'the', 'movie', 'was', 'going', 'and', 'the', 'roles', 'they', 'played', 'in', 'it', '.', 'While', 'everyone', 'hammed', 'it', 'up', ',', 'there', 'was', 'no', 'one', '-', 'upmanship', '.', 'Kline', 'showed', 'wit', 'and', 'great', 'physical', 'comedy', ',', 'Goldberg', 'and', 'Downey', 'knew', 'how', 'to', 'carry', 'on', 'a', 'funny', 'conversation', 'while', 'someone', 'else', 'was', 'talking', ',', 'I', 'could', 'just', 'go', 'on.<br', '/><br', '/>Do', 'not', 'pass', 'this', 'movie', 'by', '!'], 'label': 'pos'}


* Create the validation set from our training set.

In [0]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

* View how many examples are in each split.

In [0]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


* Build the vocabulary on the training set

In [0]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

* Why is the vocab size 25002 and not 25000? 
* One of the addition tokens is the <unk> token and the other is a <pad> token. <br>



In [0]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


* View the most common words in the vocabulary and their frequencies.

In [0]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 203459), (',', 192975), ('.', 164951), ('and', 109566), ('a', 109524), ('of', 100784), ('to', 93643), ('is', 76277), ('in', 61706), ('I', 54550), ('it', 53702), ('that', 49563), ('"', 44324), ("'s", 43067), ('this', 42411), ('-', 37120), ('/><br', 35602), ('was', 35107), ('as', 30575), ('with', 29913)]


* See the vocabulary directly using either the stoi (string to int) or itos (int to string) method.

In [0]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


* Check the labels, ensuring 0 is for negative and 1 is for positive.

In [0]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f21237b16a8>, {'neg': 0, 'pos': 1})


* Preparing the data is creating the iterators

In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

## Build the Model

* Building the model that we'll eventually train and evaluate.


In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

* Create an instance of our RNN class.

* The input dimension is the dimension of the one-hot vectors, which is equal to the vocabulary size.

* The embedding dimension is the size of the dense word vectors.
* The hidden dimension is the size of the hidden states.

* The output dimension is usually the number of classes

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

* Create a function that will tell us how many trainable parameters our model has so we can compare the number of parameters across different models.

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


##Train the Model
* Set up the training and then train the model

* Create an optimizer
* Use stochastic gradient descent (SGD)
* Learning rate

In [0]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

* Define our loss function. 
* In PyTorch this is commonly called a criterion.

* The loss function here is binary cross entropy with logits.

* The BCEWithLogitsLoss criterion carries out both the sigmoid and the binary cross entropy steps.

In [0]:
criterion = nn.BCEWithLogitsLoss()

* Place the model and the criterion on the GPU (if we have one).

In [0]:
model = model.to(device)
criterion = criterion.to(device)

* Criterion function calculates the loss 

* This rounds any value greater than 0.5 to 1 (a positive sentiment) and the rest to 0 (a negative sentiment).


In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

* The `train` function iterates over all examples, one batch at a time.


In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

* `evaluate` is similar to `train`, with a few modifications as you don't want to update the parameters when evaluating.

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

* Create a function to tell us how long an epoch takes to compare training times between models.

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

* Train the model through multiple epochs, an epoch being a complete pass through all examples in the training and validation sets.

In [0]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 6s
	Train Loss: 0.694 | Train Acc: 50.17%
	 Val. Loss: 0.697 |  Val. Acc: 50.59%
Epoch: 02 | Epoch Time: 1m 6s
	Train Loss: 0.693 | Train Acc: 49.92%
	 Val. Loss: 0.697 |  Val. Acc: 50.90%
Epoch: 03 | Epoch Time: 1m 6s
	Train Loss: 0.693 | Train Acc: 49.85%
	 Val. Loss: 0.697 |  Val. Acc: 51.05%
Epoch: 04 | Epoch Time: 1m 5s
	Train Loss: 0.693 | Train Acc: 49.92%
	 Val. Loss: 0.697 |  Val. Acc: 49.85%
Epoch: 05 | Epoch Time: 1m 6s
	Train Loss: 0.693 | Train Acc: 49.95%
	 Val. Loss: 0.697 |  Val. Acc: 51.15%


* The loss is not really decreasing and the accuracy is poor

In [0]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.712 | Test Acc: 46.08%
